**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [81]:
%pip install scikit-video

In [0]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function act performs a greedy exploration. With probability $\epsilon$ it chooses a random action (Exploration) and with probability $1 - \epsilon$ the best action according to its learned policy (Exploitation). Without $\epsilon$ there is no exploration and we may not find the optimal strategy for a Q-learning algorithm.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=30 # set small when debugging
epochs_test=30 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The board variable is a 2d array representing the rewards of every cell, namely $0.5$ is a cheese lies on the cell, $-1$ if the cell is a poisonous one, or $0$ is the rat has already visited the cell. The variable position is a 2d array made to extend the board with regard of the visibility of the rat. A value of -1 indicates we are no longer on the board.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(0, self.n_action)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def test(agent, env, epochs, prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        # For every epoch we reset the environment
        state = env.reset()

        game_over = False
        
        win = 0
        lose = 0
        # As long as we have not finished the game we continue
        while not game_over:
            prev_state = state
            
            action = agent.act(prev_state)
            state, reward, game_over = env.act(action)
        
            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
        
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose
        
        print("Win/lose count {:.2f}/{:.2f}. Average score ({:.2f})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [89]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 7.00/7.00. Average score (0.00)
Win/lose count 12.00/17.00. Average score (-2.50)
Win/lose count 9.50/13.00. Average score (-2.83)
Win/lose count 7.50/26.00. Average score (-6.75)
Win/lose count 8.50/10.00. Average score (-5.70)
Win/lose count 11.50/15.00. Average score (-5.33)
Win/lose count 8.50/18.00. Average score (-5.93)
Win/lose count 8.50/4.00. Average score (-4.62)
Win/lose count 11.00/17.00. Average score (-4.78)
Win/lose count 13.00/18.00. Average score (-4.80)
Win/lose count 9.00/23.00. Average score (-5.64)
Win/lose count 10.00/14.00. Average score (-5.50)
Win/lose count 9.50/15.00. Average score (-5.50)
Win/lose count 9.00/10.00. Average score (-5.18)
Win/lose count 9.50/12.00. Average score (-5.00)
Win/lose count 9.00/16.00. Average score (-5.12)
Win/lose count 12.00/17.00. Average score (-5.12)
Win/lose count 8.50/10.00. Average score (-4.92)
Win/lose count 8.50/18.00. Average score (-5.16)
Win/lose count 14.00/19.00. Average score (-5.15)
Win/lose count 9

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




__Answer__ :

We have that:
\begin{equation*}
Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \\
= E_{p^{\pi}}[r(s,a)  + \sum_{t \geq 1}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \\
= r(s,a)  + \gamma E_{p^{\pi}}[\sum_{t \geq 1}\gamma^{t-1}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \\
= r(s,a)  + \gamma E_{(s',a')\sim p(.|s,a)}[\sum_{t \geq 0}\gamma^{t}r(s_{t},a_{t})|s_{0}=s^{'},a_{0}=a^{'}] \\
= E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

And that:
\begin{equation*}
  Q^{*}(s,a) = \max_{\pi} Q^\pi(s,a) = Q^{\pi^{*}}(s,a) \\
  = r(s,a) + \gamma E_{(s',a')\sim p^{\pi^{*}}(.|s,a)} [ Q^{\pi^{*}}(s^{'},a^{'})] \\
  = r(s,a) + \gamma E_{s' \sim p^{\pi^{*}}(.|s,a)} [ \max_{a^{'}} Q^{*}(s^{'},a^{'})]
\end{equation*} since the optimal policy is deterministic and selects actions. Which gives the two equations. The difference between the two sides of the last equations measures the gap that we obtain by the fact that our policy is not optimal. Hence this is a good objective to assess the quality of a policy.

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
import random

class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        if len(self.memory) == 100:
          del self.memory[0]
        self.memory.append(m)

    def random_access(self):
        return random.choices(self.memory)

***
The pipeline we will use for training is given below:

In [0]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [0]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(s[None]))

    def reinforce(self, s_, n_s_, a_, r_, game_over_):

        # Two steps: first memorize the states, second learn from the pool
        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        # Create variables for keeping track of the input, target, next states and game overs
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        n_s_arr = np.zeros((self.batch_size, 5,5,self.n_state))
        game_over_arr = []
        actions_arr = []
        rewards_arr = []
        
        for i in range(self.batch_size):

            # Sample from the buffer
            s_, n_s_, a_, r_, game_over_ = self.memory.random_access()[0]
            
            # Construct the batch of input spaces / next states / actions / rewards and game overs
            input_states[i] = s_
            n_s_arr[i] = n_s_
            actions_arr.append(a_)
            rewards_arr.append(r_)
            game_over_arr.append(game_over_)

        # Compute the predicted Q function for our state and the next one
        target_q[:] = self.model.predict(input_states)
        q_prediction = self.model.predict(n_s_arr)

        for i in range(self.batch_size):
            if game_over_:
                # We are at the last state, there is no bootstrap
                target_q[i, actions_arr[i]] = rewards_arr[i]
            else:
                # We bootstrap the Q value for estimating the q value of the next state
                target_q[i, actions_arr[i]] = rewards_arr[i] + self.discount * q_prediction[i].max()
        

        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        size = args[0]
        hid_size = 30
        
        # Board seen by the rat
        input_shape = (5, 5, self.n_state)

        model = Sequential()
        model.add(keras.layers.Flatten(input_shape=input_shape))
        model.add(Dense(hid_size, activation='relu'))
        model.add(Dense(hid_size, activation='relu'))
        model.add(Dense(self.n_action)) # Output of n_action actions

        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

In [93]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, 150, prefix='fc_train')

Epoch 000/150 | Loss 0.0001 | Win/lose count 3.5/4.0 (-0.5)
Epoch 001/150 | Loss 0.0030 | Win/lose count 0.5/4.0 (-3.5)
Epoch 002/150 | Loss 0.0015 | Win/lose count 3.0/3.0 (0.0)
Epoch 003/150 | Loss 0.0078 | Win/lose count 3.0/1.0 (2.0)
Epoch 004/150 | Loss 0.0017 | Win/lose count 4.5/2.0 (2.5)
Epoch 005/150 | Loss 0.0009 | Win/lose count 1.0/3.0 (-2.0)
Epoch 006/150 | Loss 0.0002 | Win/lose count 1.5/1.0 (0.5)
Epoch 007/150 | Loss 0.0012 | Win/lose count 7.0/6.0 (1.0)
Epoch 008/150 | Loss 0.0017 | Win/lose count 2.5/1.0 (1.5)
Epoch 009/150 | Loss 0.0011 | Win/lose count 6.0/4.0 (2.0)
Epoch 010/150 | Loss 0.0033 | Win/lose count 4.5/3.0 (1.5)
Epoch 011/150 | Loss 0.0069 | Win/lose count 3.5/2.0 (1.5)
Epoch 012/150 | Loss 0.0049 | Win/lose count 2.0/3.0 (-1.0)
Epoch 013/150 | Loss 0.0045 | Win/lose count 7.0/3.0 (4.0)
Epoch 014/150 | Loss 0.0015 | Win/lose count 4.5/3.0 (1.5)
Epoch 015/150 | Loss 0.0152 | Win/lose count 4.5/3.0 (1.5)
Epoch 016/150 | Loss 0.0015 | Win/lose count 3.0/3.0

In [94]:
HTML(display_videos('fc_train130.mp4'))

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        ## Shape of the visible board 
        input_shape = (5, 5, self.n_state)
        
        # We use three layer convolutional network
        model = Sequential([
            Conv2D(32, 3, input_shape=input_shape, activation='relu', padding="same"),
            Conv2D(64, 3, activation='relu', padding="same"),
            Conv2D(32, 3, activation='relu', padding="same"),
            keras.layers.Flatten(),
            Dense(self.n_action)  # output
        ])
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [96]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env, 150,prefix='cnn_train')

Epoch 000/150 | Loss 0.0001 | Win/lose count 0.5/0 (0.5)
Epoch 001/150 | Loss 0.0001 | Win/lose count 2.5/1.0 (1.5)
Epoch 002/150 | Loss 0.0002 | Win/lose count 0.5/2.0 (-1.5)
Epoch 003/150 | Loss 0.0020 | Win/lose count 3.5/7.0 (-3.5)
Epoch 004/150 | Loss 0.0054 | Win/lose count 2.5/3.0 (-0.5)
Epoch 005/150 | Loss 0.0108 | Win/lose count 8.5/5.0 (3.5)
Epoch 006/150 | Loss 0.0133 | Win/lose count 5.5/6.0 (-0.5)
Epoch 007/150 | Loss 0.0015 | Win/lose count 5.0/6.0 (-1.0)
Epoch 008/150 | Loss 0.0009 | Win/lose count 6.0/2.0 (4.0)
Epoch 009/150 | Loss 0.0006 | Win/lose count 12.0/4.0 (8.0)
Epoch 010/150 | Loss 0.0036 | Win/lose count 12.5/8.0 (4.5)
Epoch 011/150 | Loss 0.0003 | Win/lose count 8.5/1.0 (7.5)
Epoch 012/150 | Loss 0.0036 | Win/lose count 6.5/3.0 (3.5)
Epoch 013/150 | Loss 0.0003 | Win/lose count 10.5/0 (10.5)
Epoch 014/150 | Loss 0.0020 | Win/lose count 6.5/6.0 (0.5)
Epoch 015/150 | Loss 0.0003 | Win/lose count 6.0/2.0 (4.0)
Epoch 016/150 | Loss 0.0002 | Win/lose count 9.5/2.

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [97]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 19.00/0.00. Average score (19.00)
Win/lose count 23.00/3.00. Average score (19.50)
Win/lose count 15.50/3.00. Average score (17.17)
Win/lose count 6.50/2.00. Average score (14.00)
Win/lose count 10.50/2.00. Average score (12.90)
Win/lose count 17.50/5.00. Average score (12.83)
Win/lose count 14.00/3.00. Average score (12.57)
Win/lose count 15.50/2.00. Average score (12.69)
Win/lose count 14.50/3.00. Average score (12.56)
Win/lose count 14.00/4.00. Average score (12.30)
Win/lose count 12.50/1.00. Average score (12.23)
Win/lose count 14.00/4.00. Average score (12.04)
Win/lose count 16.00/3.00. Average score (12.12)
Win/lose count 4.50/2.00. Average score (11.43)
Win/lose count 11.50/2.00. Average score (11.30)
Win/lose count 18.00/0.00. Average score (11.72)
Win/lose count 11.00/4.00. Average score (11.44)
Win/lose count 14.50/5.00. Average score (11.33)
Win/lose count 17.00/2.00. Average score (11.53)
Win/lose count 9.00/4.00. Average score (11.20)
Win/los

In [98]:
HTML(display_videos('cnn_test10.mp4'))

In [99]:
HTML(display_videos('fc_test10.mp4'))

__Answer__: The test function evaluate our learned agents by using the greedy policy consisting on choosing the best actions according to the Q function at every steps. The behaviour that we typically encounter in this scenario is that the agent get stucks in loop when it is in a difficult state, for instance when there is no cheese in sight. This behaviour is even more present when we use a model of fully-connected layers.

The temperature of the environment dictates the amount of cheese and poisonous cell (cheese overriding poisonous cell). When the temperature is low, bonus and malus are both scarce and the model has more difficulty to learn an appropriate behaviour. In this case, both during training and testing we end up with a more pronounced behaviour towards agents stuck in loops. According to our tests the cnn model seems to perform better but is also harder to train.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [0]:
def train_explore(agent, env, epoch, prefix=''):
    # Number of won games
    score = 0
    loss = 0
    ini_eps = agent.epsilon
    
    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False
        
        win = 0
        lose = 0
        pos_reg = 0
        
        while not game_over:
            # The agent performs an action
            action = agent.act(state)
            
            prev_state = state
            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            state, reward, game_over = env.act(action)
            
            
            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state, action, reward, game_over)
          
            
            pos_reg += env.last_pos_reg
            reward_without_malus = reward + env.last_pos_reg

            # Update the counter
            if reward_without_malus > 0:
                win = win + reward_without_malus
            if reward_without_malus < 0:
                lose = lose - reward_without_malus
        
        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {:.2f}/{:.2f} ({:.2f})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

    
class EnvironmentExploring(Environment):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1, reg_position = 0.1):
        super(EnvironmentExploring, self).__init__(grid_size=grid_size, max_time=max_time, temperature=temperature)
        self.reg_position = reg_position ## Parameter for the amount of malus we put on visited cells
        self.pos_reg = np.zeros((self.grid_size, self.grid_size)) ## New feature consisting of the malus on visited cells
        self.last_pos_reg = 0 ## Malus of the last visited cell

    def reset(self):

        s_ = super(EnvironmentExploring, self).reset()

        # No cell has been visited yet
        self.pos_reg[:] = 0.
    
        state = np.concatenate((self.pos_reg.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]
        return state

    def act(self,action):


        state, reward, game_over = super(EnvironmentExploring, self).act(action)
        if train:
            ## While training, the visiting seen cells have a cost
            reward = reward - self.pos_reg[self.x, self.y]
        self.last_pos_malus = self.pos_reg[self.x, self.y]
        self.pos_reg[self.x, self.y] = self.reg_position

        # We add the feature of the visits on the state
        state = np.concatenate((self.pos_reg.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        
        # Window arround the cell
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]
        return state, reward, game_over

    ##Put black cell for visited one.
    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.pos_reg > 0, 0] = 0
        b[self.pos_reg > 0, 1] = 0
        b[self.pos_reg > 0, 2] = 0
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b



In [101]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.2, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, 150, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/150 | Loss 0.0118 | Win/lose count 9.50/22.80 (-13.30)
Epoch 001/150 | Loss 0.0081 | Win/lose count 10.50/25.10 (-14.60)
Epoch 002/150 | Loss 0.0044 | Win/lose count 16.50/18.50 (-2.00)
Epoch 003/150 | Loss 0.0026 | Win/lose count 13.00/22.00 (-9.00)
Epoch 004/150 | Loss 0.0068 | Win/lose count 12.00/22.80 (-10.80)
Epoch 005/150 | Loss 0.0091 | Win/lose count 11.00/15.30 (-4.30)
Epoch 006/150 | Loss 0.0066 | Win/lose count 11.00/19.80 (-8.80)
Epoch 007/150 | Loss 0.0061 | Win/lose count 8.00/21.50 (-13.50)
Epoch 008/150 | Loss 0.0184 | Win/lose count 17.00/20.60 (-3.60)
Epoch 009/150 | Loss 0.0024 | Win/lose count 17.00/17.70 (-0.70)
Epoch 010/150 | Loss 0.0046 | Win/lose count 17.50/15.60 (1.90)
Epoch 011/150 | Loss 0.0132 | Win/lose count 14.50/20.40 (-5.90)
Epoch 012/150 | Loss 0.0067 | Win/lose count 18.00/15.60 (2.40)
Epoch 013/150 | Loss 0.0038 | Win/lose count 22.00/16.40 (5.60)
Epoch 014/150 | Loss 0.0044 | Win/lose count 15.00/19.00 (-4.00)
Epoch 015/150 | Loss 0.011

In [102]:
HTML(display_videos('cnn_train_explore10.mp4'))

In [103]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')

Win/lose count 20.00/16.20. Average score (3.80)
Win/lose count 17.00/17.60. Average score (1.60)
Win/lose count 16.00/18.50. Average score (0.23)
Win/lose count 13.00/20.30. Average score (-1.65)
Win/lose count 21.00/15.10. Average score (-0.14)
Win/lose count 8.00/18.00. Average score (-1.78)
Win/lose count 23.50/20.90. Average score (-1.16)
Win/lose count 23.00/17.80. Average score (-0.36)
Win/lose count 21.00/15.50. Average score (0.29)
Win/lose count 15.50/16.10. Average score (0.20)
Win/lose count 10.00/18.50. Average score (-0.59)
Win/lose count 18.50/17.80. Average score (-0.48)
Win/lose count 12.00/24.10. Average score (-1.38)
Win/lose count 16.00/21.20. Average score (-1.65)
Win/lose count 22.00/19.60. Average score (-1.38)
Win/lose count 21.00/18.30. Average score (-1.13)
Win/lose count 18.00/18.10. Average score (-1.06)
Win/lose count 20.50/17.80. Average score (-0.86)
Win/lose count 30.00/12.10. Average score (0.13)
Win/lose count 24.00/15.00. Average score (0.57)
Win/lose

In [104]:
HTML(display_videos('cnn_test_explore10.mp4'))

Thanks to these modifications, we obtain better performances. The agent effectively avoid getting back to previous states and there is almost no loops, hence resulting in a better exploration.

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

In [0]:
def generate_games(agent, env, nTrain):
    ## The aim of the function is to generate samples of behaviour that produces rewards

    #Array of the states chosen
    st = np.zeros((nTrain,5,5,3))
    countStates = 0
    # Corresponding actions
    ac = np.zeros((nTrain, 4))
    counter_attempts = 0
    num_samples = 0

    while countStates < nTrain:
        # initialize the game
        s_ = env.reset()
        game_over = False
        
        while not game_over:
            a_ = agent.act(s_, False)
            n_s_, r_, game_over = env.act(a_)
            s_ = n_s_
            if countStates < nTrain and r_ > 0:
                
                st[countStates] = s_
                ac[countStates, a_] = 1
                countStates += 1
    return st, ac

In [0]:
train_size = 1000
data_train = generate_games(agent, env, train_size)
val_size = 100
data_val = generate_games(agent, env, val_size)
test_size = 100
data_test = generate_games(agent, env, test_size)

In [0]:
inp_shape = (5, 5, 3)
#We take the same model as before
model = Sequential([
            Conv2D(32, 3, input_shape=inp_shape, activation='relu', padding="same"),
            Conv2D(64, 3, activation='relu', padding="same"),
            Conv2D(32, 3, activation='relu', padding="same"),
            keras.layers.Flatten(),
            Dense(4, activation="softmax")  # output
        ])
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=sgd(lr=.1, decay=1e-4, momentum=0.0),
              metrics=['accuracy'])

In [108]:
model.fit(
    data_train[0], data_train[1],
    batch_size=32,
    validation_data=data_val,
    epochs=50
)

Train on 1000 samples, validate on 100 samples
Epoch 1/50
1000/1000 [==============================] - 1s 978us/step - loss: 1.1611 - acc: 0.5460 - val_loss: 0.5510 - val_acc: 0.9100
Epoch 2/50
1000/1000 [==============================] - 0s 153us/step - loss: 0.1332 - acc: 0.9940 - val_loss: 0.0173 - val_acc: 1.0000
Epoch 3/50
1000/1000 [==============================] - 0s 160us/step - loss: 0.0156 - acc: 0.9990 - val_loss: 0.0068 - val_acc: 1.0000
Epoch 4/50
1000/1000 [==============================] - 0s 159us/step - loss: 0.0099 - acc: 0.9990 - val_loss: 0.0050 - val_acc: 1.0000
Epoch 5/50
1000/1000 [==============================] - 0s 142us/step - loss: 0.0078 - acc: 0.9990 - val_loss: 0.0025 - val_acc: 1.0000
Epoch 6/50
1000/1000 [==============================] - 0s 154us/step - loss: 0.0069 - acc: 0.9990 - val_loss: 0.0024 - val_acc: 1.0000
Epoch 7/50
1000/1000 [==============================] - 0s 158us/step - loss: 0.0053 - acc: 0.9990 - val_loss: 0.0015 - val_acc: 1.0000
E

In [0]:
# We create a basic agent that follows a given model
class AgentMimic(Agent):
    def __init__(self, model, grid_size, epsilon=0.1, n_state=2):
        super(ImitationAgent, self).__init__(epsilon=epsilon)
        self.model = model ## Model that we trained previously
        self.grid_size = grid_size
        self.n_state = n_state
        
        
    def learned_act(self, state):
        # We just take action according to the trained model
        action_distrib = self.model.predict(state[None])
        return action_distrib.max()

In [110]:
nv = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
mimic_agent = AgentMimic(model, size, epsilon=0.1, n_state=3)
test(agent, env, 20, prefix="imitation_")

Win/lose count 14.00/17.40. Average score (-3.40)
Win/lose count 8.50/21.30. Average score (-8.10)
Win/lose count 22.00/14.20. Average score (-2.80)
Win/lose count 19.50/17.50. Average score (-1.60)
Win/lose count 18.00/14.20. Average score (-0.52)
Win/lose count 21.00/17.70. Average score (0.12)
Win/lose count 16.50/17.00. Average score (0.03)
Win/lose count 11.00/20.10. Average score (-1.11)
Win/lose count 23.00/15.10. Average score (-0.11)
Win/lose count 25.00/15.30. Average score (0.87)
Win/lose count 23.50/17.60. Average score (1.33)
Win/lose count 18.50/16.70. Average score (1.37)
Win/lose count 9.00/18.70. Average score (0.52)
Win/lose count 21.00/21.10. Average score (0.47)
Win/lose count 9.00/17.50. Average score (-0.13)
Win/lose count 24.00/14.30. Average score (0.49)
Win/lose count 16.00/19.90. Average score (0.23)
Win/lose count 10.00/20.00. Average score (-0.34)
Win/lose count 21.50/15.20. Average score (0.01)
Win/lose count 23.00/19.60. Average score (0.18)
Final score: 0

We obtain a final score of 0.18 for the imitation learning model, while the original model gives a score of 11. The imitation is better than the full neural network model that we presented but lags behind the original model it is derived from.

***